In [3]:
import clang.cindex

In [36]:

def find_typerefs(node, typename, prefix=""):
    print "{}{}  {} {}: type={} \t{} {}".format(prefix, node.location.line, node.spelling, node.kind, node.type.kind, node.spelling, node.displayname) 
    definition = clang.cindex.conf.lib.clang_getCursorDefinition(node);
    if definition:
        print "DEFINITION", definition.location.line, definition.kind, definition.spelling
        print "ARGS", clang.cindex.conf.lib.clang_Cursor_getTemplateArgumentType(node,0).spelling
        #     for x in node.get_children():
#         print x
    # Recurse for children of this node
    for c in node.get_children():
        find_typerefs(c, typename, (prefix + "->"))

In [34]:
code = """

/*
namespace epicure {

template <class T>
class RPC {T t;};

template <class T, typename ...Args>
auto publish(Args && ...args) {
    return RPC<T>();
}
}
*/

template <class T> 
int foo()
{
    return 10;
}

class Dog {

};

int main()
{
    //int cow;
  //  epicure::RPC<Dog> x = epicure::publish<Dog>();
    int bar = foo<Dog>();
    //int man;
}
"""

with open("code.cpp", "w") as text_file:
    text_file.write(code)

In [37]:
index = clang.cindex.Index.create()
tu = index.parse("code.cpp")
find_typerefs(tu.cursor, "Dog")

0  code.cpp CursorKind.TRANSLATION_UNIT: type=TypeKind.INVALID 	code.cpp code.cpp
->17  foo CursorKind.FUNCTION_TEMPLATE: type=TypeKind.FUNCTIONPROTO 	foo foo()
DEFINITION 17 CursorKind.FUNCTION_TEMPLATE foo
ARGS 
->->16  T CursorKind.TEMPLATE_TYPE_PARAMETER: type=TypeKind.UNEXPOSED 	T T
DEFINITION 16 CursorKind.TEMPLATE_TYPE_PARAMETER T
ARGS 
->->18   CursorKind.COMPOUND_STMT: type=TypeKind.INVALID 	 
->->->19   CursorKind.RETURN_STMT: type=TypeKind.INVALID 	 
->->->->19   CursorKind.INTEGER_LITERAL: type=TypeKind.INT 	 
->22  Dog CursorKind.CLASS_DECL: type=TypeKind.RECORD 	Dog Dog
DEFINITION 22 CursorKind.CLASS_DECL Dog
ARGS 
->26  main CursorKind.FUNCTION_DECL: type=TypeKind.FUNCTIONPROTO 	main main()
DEFINITION 26 CursorKind.FUNCTION_DECL main
ARGS 
->->27   CursorKind.COMPOUND_STMT: type=TypeKind.INVALID 	 
->->->30   CursorKind.DECL_STMT: type=TypeKind.INVALID 	 
->->->->30  bar CursorKind.VAR_DECL: type=TypeKind.INT 	bar bar
DEFINITION 30 CursorKind.VAR_DECL bar
ARGS 
->->->->-

In [87]:
x = tu.cursor.type.get_named_type().spelling

In [72]:
clang.cindex.conf.lib.

''

In [1]:
import clang.cindex
from clang.cindex import *
import cymbal
from ctypes import *

cymbal.monkeypatch_type('get_template_argument_type',
                        'clang_Type_getTemplateArgumentAsType',
                        [Type, c_uint],
                        Type)

cymbal.monkeypatch_type('get_num_template_arguments',
                        'clang_Type_getNumTemplateArguments',
                        [Type],
                        c_int)

# check if the cursor's type is a template
def is_template(node):
    return hasattr(node, 'type') and node.type.get_num_template_arguments() != -1



In [2]:
source = """
//#include <string>
//#include <iostream>

template <typename T> class X {};
X<bool> x;

template <class T>
int foo(int y) {return 10;}

template <class A, class B>
int baz(int y) {return 10;}

template <class T>
void publish() {;}

class Dog {
public:
    int bark(double volume, const int& language);
    void run(float speed) const {  }
};
class Cat {};

int main() {
    int x = foo<Dog>(7);
    int y = baz<Dog,Cat>(7);
    publish<Cat>();
    publish<Dog>();
    
}
"""

In [10]:
index = clang.cindex.Index.create()

# parse the TU
tu = clang.cindex.TranslationUnit.from_source('t.cpp', ['-std=c++11'], unsaved_files=[('t.cpp', source)])
for x in tu.diagnostics:
    print x

for c in tu.cursor.walk_preorder():
    if True: # is_template(c):
        t = c.type
        print "A", c.location.line, "usr:", c.get_usr(), c.displayname, c.spelling, c.kind, t.kind, t.spelling, t.get_num_template_arguments()
        print "B", t.get_template_argument_type(0).spelling
        sp = c.semantic_parent
        lp = c.lexical_parent
        d = c.get_definition()
        print "D", sp.displayname if sp else "None", lp.displayname if lp else "None", d.location.line if d else "No definition", d.get_usr() if d else "No usr"
        for x in c.get_arguments():
            print "C", x.type.spelling

A 0 usr:  t.cpp t.cpp CursorKind.TRANSLATION_UNIT TypeKind.INVALID  -1
B 
D None None No definition No usr
A 5 usr: c:@ST>1#T@X X<T> X CursorKind.CLASS_TEMPLATE TypeKind.INVALID  -1
B 
D t.cpp t.cpp 5 c:@ST>1#T@X
A 5 usr: c:t.cpp@54 T T CursorKind.TEMPLATE_TYPE_PARAMETER TypeKind.UNEXPOSED T -1
B 
D X<T> X<T> 5 c:t.cpp@54
A 6 usr: c:@x x x CursorKind.VAR_DECL TypeKind.UNEXPOSED X<bool> 1
B bool
D t.cpp t.cpp 6 c:@x
A 6 usr:  X X CursorKind.TEMPLATE_REF TypeKind.INVALID  -1
B 
D None None 5 c:@ST>1#T@X
A 6 usr:  X X CursorKind.CALL_EXPR TypeKind.UNEXPOSED X<bool> 1
B bool
D x None 5 c:@S@X>#b@F@X#
A 9 usr: c:@FT@>1#Tfoo#I#I# foo(int) foo CursorKind.FUNCTION_TEMPLATE TypeKind.FUNCTIONPROTO int (int) -1
B 
D t.cpp t.cpp 9 c:@FT@>1#Tfoo#I#I#
A 8 usr: c:t.cpp@100 T T CursorKind.TEMPLATE_TYPE_PARAMETER TypeKind.UNEXPOSED T -1
B 
D foo(int) foo(int) 8 c:t.cpp@100
A 9 usr: c:t.cpp@117@FT@>1#Tfoo#I#I#@y y y CursorKind.PARM_DECL TypeKind.INT int -1
B 
D foo(int) foo(int) 9 c:t.cpp@117@FT@>1#Tfoo

In [4]:
import clang.cindex
from clang.cindex import *
import cymbal
from ctypes import *

cymbal.monkeypatch_type('get_template_argument_type',
                        'clang_Type_getTemplateArgumentAsType',
                        [Type, c_uint],
                        Type)

cymbal.monkeypatch_type('get_num_template_arguments',
                        'clang_Type_getNumTemplateArguments',
                        [Type],
                        c_int)                
        

In [137]:
def find_all_published_types(cursor):
    types = []
    for node in cursor.walk_preorder():
        if node.type.kind == TypeKind.FUNCTIONPROTO and node.spelling == "publish":
            for child in node.get_children():
                if child.type.kind == TypeKind.RECORD:
                    d = child.get_definition()
                    if d.kind in [CursorKind.CLASS_DECL, CursorKind.STRUCT_DECL]:
                        types.append(d)
    return types

In [93]:
def find_definition(cursor, target_type_spelling):
    for node in cursor.walk_preorder():
        if node.kind == CursorKind.CLASS_DECL and node.type.spelling == target_type_spelling:
                return node

In [128]:
def describe_methods(definition_cursor):
    methods = {}
    # TODO: check for class or struct definition
    for node in definition_cursor.walk_preorder():
        if node.kind == CursorKind.CXX_METHOD:
            methods[node.spelling] = {
                "args": [{"name": arg.spelling, "type": arg.type.spelling} for arg in node.get_arguments()],
                "return": node.result_type.spelling,
            }
    return methods

In [12]:
import clang.cindex
from clang.cindex import CursorKind, TypeKind

def is_call_to_publish(node):
    #TODO: WATCH OUT FOR THIS. NEED A MUCH BETTER CHECK.
    return node.type.kind == TypeKind.FUNCTIONPROTO and node.spelling == "publish"

def find_all_published_type_definitions(cursor):
    definitions = []
    for node in cursor.walk_preorder():
        if is_call_to_publish(node):
            for child in node.get_children():
                if child.type.kind == TypeKind.RECORD:
                    d = child.get_definition()
                    if d.kind in [CursorKind.CLASS_DECL, CursorKind.STRUCT_DECL]:
                        definitions.append(d)
    return definitions

def describe_methods(definition_cursor):
    methods = {}
    for node in definition_cursor.walk_preorder():
        if node.kind == CursorKind.CXX_METHOD:
            methods[node.spelling] = {
                "args": [{"name": arg.spelling, "type": arg.type.spelling} for arg in node.get_arguments()],
                "return": node.result_type.spelling,
            }
    return methods

In [13]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

# parse the TU
tu = clang.cindex.TranslationUnit.from_source('t.cpp', ['-std=c++11'], unsaved_files=[('t.cpp', source)])
for x in tu.diagnostics:
    print x

types = find_all_published_type_definitions(tu.cursor)
for t in types:
    print t.location.line, t.spelling
    print t.get_definition().location.line, t.get_definition().spelling

for t in types:
#     d = find_definition(tu.cursor, t.type.spelling)
#     print d.location.line, d.type.spelling
    m = describe_methods(t)
    pp.pprint(m)

22 Cat
22 Cat
17 Dog
17 Dog
{   }
{   'bark': {   'args': [   {   'name': 'volume', 'type': 'double'},
                            {   'name': 'language', 'type': 'const int &'}],
                'return': 'int'},
    'run': {   'args': [{   'name': 'speed', 'type': 'float'}], 'return': 'void'}}


In [34]:
"{name}, {a}, {b}".format(name="Dog", **{"a":"boy", "b":"man", "c":"cow"})

'Dog, boy, man'

In [36]:
from string import Template

def generate_request_method(target):
    return "void foo();"

def generate_service_class(target):
    code = Template(
    '''
    struct ${name}ServiceImpl : public RPCService {
        ${request_method}
    };
    '''
    )
    return code.substitute(**{
        "name": target["name"],
        "request_method": generate_request_method(target)
    })

target = {"name": "Dog"}
print generate_service_class(target)


    struct DogServiceImpl : public RPCService {
        void foo();
    };
    


In [27]:
'''line {0}
line {1}
line {2}'''.format(1,2,3)

'line 1\nline 2\nline 3'

In [38]:
"".join(["dog","cat","man"])

'dogcatman'